# ABOVE AC9 Data Processing  

## Vicarious calibration (subtracting averaged filtered spectra from averaged unfiltered spectra)

**Authors:** Catherine Kuhn and Anna Simpson
**Last Updated:** August, 30th, 2018

In [1]:
### Import the required python libraries
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import glob
import os
import sys
import csv
from scipy import interpolate
from os import listdir
from os.path import isfile, join
import matplotlib.gridspec as gridspec

### Functions

In [5]:
def get_subdirectories(directory=os.getcwd()):
    if directory in os.getcwd():
        directories = os.listdir(os.getcwd())
    else:
        directories = os.listdir(directory)
    directories = [subdirectory for subdirectory in directories if not subdirectory.startswith('.')]
    return directories

In [48]:
def plotting_vicarious_calibrations(list_of_files, title='Title'):
    j = 0
    x = []
    y = []
    labels_list = []
    for filename in range(len(list_of_files)):   
            df = pd.read_csv(list_of_files[filename], skiprows = 0, delimiter= '\t')         
            df.reset_index(inplace=True, drop=False)
            df.rename(columns = {'c_mean':'mean', 'a_mean':'mean', 'c_std':'std','a_std':'std'}, inplace = True)
            Sensor, Date, Site, Sample_Type, Abs_Type, Rep, T1, T2= list_of_files[filename].split('/')[-1].split('.')[0].split('_')
            labelname = Site
            labels_list.append(labelname)
            plt.plot('wl','mean',data=df,label=labelname)
            #plt.scatter('wl', 'a_mean', data = df)
            plt.errorbar('wl', 'mean', yerr='std', fmt='k-', linewidth=0.5, data = df) 
    plt.ylabel('a [1/m]')
    plt.title(title)
    plt.legend(labels_list)
    return plt

## Inputs

In [4]:
averaged_files_name = '3_averaged_summary_stats'
vicariously_calibrated_files_name = '4_vicarious_calibrations'

averaged_files_location = os.path.abspath('..')+'/Data/'+averaged_files_name
vicariously_calibrated_files_location = os.path.abspath('..')+'/Data/'+vicariously_calibrated_files_name

## Subtract unfiltered files from raw files

In [27]:
## Create directory to store vicariously calibrated files
if not os.path.exists(vicariously_calibrated_files_location):
    os.makedirs(vicariously_calibrated_files_location)
    
dict_of_files = {}
for filename in sorted(glob.glob(averaged_files_location+'/*.csv')):
    #Get file info
    basename = filename.split('/')[-1].split('.')[0]
    if 'cal' in basename or 'Calibrations' in basename:
        continue
    else:
        Sensor, Date, Site, Sample_Type, Abs_Type, filetype= filename.split('/')[-1].split('.')[0].split('_')
        sample_id_for_calibration = '_'.join([Date, Site, Abs_Type])
        if sample_id_for_calibration not in dict_of_files:
            dict_of_files[sample_id_for_calibration] = [filename]
        else:
            dict_of_files[sample_id_for_calibration].append(filename)

In [56]:
dict_of_corrected_values = {}
for i in dict_of_files:
    for j in dict_of_files[i]:
        if 'raw' in j:
            raw_file = pd.read_csv(j, skiprows=0, delimiter = '\t')
            raw_file.rename(columns = {'c_mean':'mean', 'a_mean':'mean', 'c_std':'std','a_std':'std'}, inplace = True)
        if 'fil' in j:
            fil_file = pd.read_csv(j, skiprows=0, delimiter = '\t')
            fil_file.rename(columns = {'c_mean':'mean', 'a_mean':'mean', 'c_std':'std','a_std':'std'}, inplace = True)
    new_df = raw_file[['wl']]
    new_df['corrected'] = raw_file[['mean']]-fil_file[['mean']]
    output_file_name = "AC9"+'_'+i
    outputdir = vicariously_calibrated_files_location + '/' + output_file_name + '_viccal.csv'
    new_df.to_csv(outputdir, sep='\t')

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
